In [1]:
from itertools import combinations
from nba_api.stats.endpoints import commonteamroster, teamdashlineups
from nba_api.stats.static import teams
import pandas as pd
import time

In [2]:
def get_team_roster(team_name: str, season: str):
    nba_teams = teams.get_teams()
    team = next((team for team in nba_teams if team['full_name'].lower() == team_name.lower()), None)
    if not team:
        raise ValueError(f"Team '{team_name}' not found.")
    
    team_id = team['id']
    roster_data = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
    roster = roster_data.get_normalized_dict()
    roster_df = pd.DataFrame(roster['CommonTeamRoster'])
    return roster_df

In [3]:
def normalize_player_name(player_name):

    parts = player_name.split()
    if len(parts) > 1:
        return f"{parts[0][0]}. {' '.join(parts[1:])}"
    return player_name

In [4]:
def exponential_backoff_retry(func, *args, max_retries=5, initial_wait=1, multiplier=2, **kwargs):
    wait_time = initial_wait
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Error: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
                wait_time *= multiplier
            else:
                raise e

In [5]:
def analyze_lineups(players, teamdashlineups, team_id, opponent_team_id):

    retries = 5
    backoff = 1
    while retries > 0:
        try:
            lineups_df = exponential_backoff_retry(
                teamdashlineups.TeamDashLineups,
                team_id=team_id,
                opponent_team_id=opponent_team_id
            ).get_data_frames()[1]
            break
        except Exception as e:
            print(f"Error fetching lineup data: {e}. Retrying in {backoff} seconds...")
            time.sleep(backoff)
            backoff *= 2
            retries -= 1

    if retries == 0:
        print("Failed to fetch lineup data after multiple retries.")
        return None

    normalized_players = [normalize_player_name(player) for player in players]

    with_players = lineups_df[
        lineups_df["GROUP_NAME"].apply(lambda x: all(player in x for player in normalized_players))
    ]

    without_players = lineups_df[
        ~lineups_df["GROUP_NAME"].apply(lambda x: all(player in x for player in normalized_players))
    ]

    total_minutes_with = with_players["MIN"].sum()
    total_plus_minus_with = with_players["PLUS_MINUS"].sum()
    
    total_minutes_without = without_players["MIN"].sum()
    total_plus_minus_without = without_players["PLUS_MINUS"].sum()

    print("Lineups with players:")
    print(f"  Total Minutes: {total_minutes_with}")
    print(f"  Total Plus-Minus: {total_plus_minus_with}")

    print("\nLineups without players:")
    print(f"  Total Minutes: {total_minutes_without}")
    print(f"  Total Plus-Minus: {total_plus_minus_without}")

    return {
        "with_players": {
            "total_minutes": total_minutes_with,
            "total_plus_minus": total_plus_minus_with,
            "lineup_count": len(with_players)
        },
        "without_players": {
            "total_minutes": total_minutes_without,
            "total_plus_minus": total_plus_minus_without,
            "lineup_count": len(without_players)
        }
    }


In [6]:
players = ["B Podziemski", "Draymond Green"]
data = analyze_lineups(players, teamdashlineups, team_id = 1610612744, opponent_team_id = 1610612738)

Lineups with players:
  Total Minutes: 0.0
  Total Plus-Minus: 0.0

Lineups without players:
  Total Minutes: 48.00000000000001
  Total Plus-Minus: 6.0


In [7]:
warriors_roster = get_team_roster("golden state warriors", "2024-25")
player_names = warriors_roster['PLAYER'].tolist()
player_names

['Gary Payton II',
 'Jonathan Kuminga',
 'Kyle Anderson',
 'Brandin Podziemski',
 'Moses Moody',
 'Kevon Looney',
 'Buddy Hield',
 'Gui Santos',
 'Quinten Post',
 'Andrew Wiggins',
 'Draymond Green',
 'Stephen Curry',
 'Trayce Jackson-Davis',
 'Lindy Waters III',
 'Pat Spencer',
 'Dennis Schröder']

In [8]:
categories = {"Duo": 2, "Trio": 3, "Quartet": 4, "Quintet": 5}
top_results = {}

for category, size in categories.items():
    print(f"\nAnalyzing {category}s:")
    combinations_list = list(combinations(player_names, size))
    results = []
    
    for combo in combinations_list:
        time.sleep(0.5)
        print(f"Analyzing {category}: {combo}")
        result = analyze_lineups(
            players=list(combo),
            teamdashlineups=teamdashlineups,
            team_id=1610612744,  # Warriors team ID
            opponent_team_id=1610612738  # Celtics team ID
        )
        if result:
            results.append({
                "group": combo,
                "plus_minus_with": result["with_players"]["total_plus_minus"],
                "minutes_with": result["with_players"]["total_minutes"]
            })

    sorted_results = sorted(results, key=lambda x: x["plus_minus_with"], reverse=True)[:5]
    top_results[category] = sorted_results

for category, results in top_results.items():
    print(f"\nTop 5 {category}s by +/-:")
    for entry in results:
        print(f"{category}: {entry['group']}, +/-: {entry['plus_minus_with']}, Minutes: {entry['minutes_with']}")


Analyzing Duos:
Analyzing Duo: ('Gary Payton II', 'Jonathan Kuminga')
Lineups with players:
  Total Minutes: 2.8833333333333333
  Total Plus-Minus: -1.0

Lineups without players:
  Total Minutes: 45.11666666666667
  Total Plus-Minus: 7.0
Analyzing Duo: ('Gary Payton II', 'Kyle Anderson')
Lineups with players:
  Total Minutes: 4.74
  Total Plus-Minus: -2.0

Lineups without players:
  Total Minutes: 43.260000000000005
  Total Plus-Minus: 8.0
Analyzing Duo: ('Gary Payton II', 'Brandin Podziemski')
Lineups with players:
  Total Minutes: 0.0
  Total Plus-Minus: 0.0

Lineups without players:
  Total Minutes: 48.00000000000001
  Total Plus-Minus: 6.0
Analyzing Duo: ('Gary Payton II', 'Moses Moody')
Lineups with players:
  Total Minutes: 2.5166666666666666
  Total Plus-Minus: 6.0

Lineups without players:
  Total Minutes: 45.483333333333334
  Total Plus-Minus: 0.0
Analyzing Duo: ('Gary Payton II', 'Kevon Looney')
Lineups with players:
  Total Minutes: 7.121666666666666
  Total Plus-Minus: -1.

In [9]:
for category, results in top_results.items():
    print(f"\nTop 5 {category}s by +/-:")
    for entry in results:
        print(f"{category}: {entry['group']}, +/-: {entry['plus_minus_with']}, Minutes: {entry['minutes_with']}")


Top 5 Duos by +/-:
Duo: ('Buddy Hield', 'Andrew Wiggins'), +/-: 22.0, Minutes: 11.754999999999999
Duo: ('Buddy Hield', 'Stephen Curry'), +/-: 21.0, Minutes: 14.923333333333334
Duo: ('Buddy Hield', 'Draymond Green'), +/-: 19.0, Minutes: 14.771666666666667
Duo: ('Kevon Looney', 'Andrew Wiggins'), +/-: 10.0, Minutes: 7.861666666666666
Duo: ('Kevon Looney', 'Stephen Curry'), +/-: 9.0, Minutes: 10.745

Top 5 Trios by +/-:
Trio: ('Buddy Hield', 'Andrew Wiggins', 'Draymond Green'), +/-: 22.0, Minutes: 11.521666666666667
Trio: ('Buddy Hield', 'Andrew Wiggins', 'Stephen Curry'), +/-: 22.0, Minutes: 11.754999999999999
Trio: ('Buddy Hield', 'Draymond Green', 'Stephen Curry'), +/-: 22.0, Minutes: 11.521666666666667
Trio: ('Kevon Looney', 'Buddy Hield', 'Andrew Wiggins'), +/-: 10.0, Minutes: 3.856666666666667
Trio: ('Kevon Looney', 'Buddy Hield', 'Draymond Green'), +/-: 10.0, Minutes: 3.6233333333333335

Top 5 Quartets by +/-:
Quartet: ('Buddy Hield', 'Andrew Wiggins', 'Draymond Green', 'Stephen C

# Takeaways
## Note: This code can be adapted to provide analytics for any specific team, it simply would just take a long time to run if i was to provide this analysis tool against every team, team specific insights also provides great feedback!
- The most obvious